<!---
Latex Macros
-->
$$
\newcommand{\bar}{\,|\,}
\newcommand{\Xs}{\mathcal{X}}
\newcommand{\Ys}{\mathcal{Y}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\weights}{\mathbf{w}}
\newcommand{\balpha}{\boldsymbol{\alpha}}
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\aligns}{\mathbf{a}}
\newcommand{\align}{a}
\newcommand{\source}{\mathbf{s}}
\newcommand{\target}{\mathbf{t}}
\newcommand{\ssource}{s}
\newcommand{\starget}{t}
\newcommand{\repr}{\mathbf{f}}
\newcommand{\repry}{\mathbf{g}}
\newcommand{\x}{\mathbf{x}}
\newcommand{\prob}{p}
\newcommand{\vocab}{V}
\newcommand{\params}{\boldsymbol{\theta}}
\newcommand{\param}{\theta}
\DeclareMathOperator{\perplexity}{PP}
\DeclareMathOperator{\argmax}{argmax}
\DeclareMathOperator{\argmin}{argmin}
\newcommand{\train}{\mathcal{D}}
\newcommand{\counts}[2]{\#_{#1}(#2) }
\newcommand{\length}[1]{\text{length}(#1) }
\newcommand{\indi}{\mathbb{I}}
$$

## Hints

A non-exhaustive list of things you might want to give a try:
- better tokenization
- experiment with pre-trained word representations such as [word2vec](https://code.google.com/archive/p/word2vec/), or [GloVe](http://nlp.stanford.edu/projects/glove/). Be aware that these representations might take a lot of parameters in your model. Be sure you use only the words you expect in the training/dev set and account for OOV words. When saving the model parameters, pre-rained word embeddings can simply be used in the word embedding matrix of your model. As said, make sure that this word embedding matrix does not contain all of word2vec or GloVe. Your submission is limited, and we will not allow uploading nor using the whole representations set (up to 3GB!)
- reduced sizes of word representations
- bucketing and batching (our implementation is deliberately not a good one!)
  - make sure to draw random batches from the data! (we do not provide this in our code!)
- better models:
  - stacked RNNs (see tf.contrib.rnn.MultiRNNCell)
  - bi-directional RNNs
  - attention
  - word-by-word attention
  - conditional encoding
  - get model inspirations from papers on [nlp.stanford.edu/projects/snli/](nlp.stanford.edu/projects/snli/)
  - sequence-to-sequence encoder-decode architecture for producing the right ordering
- better training procedure:
  - different training algorithms
  - dropout on the input and output embeddings (see tf.nn.dropout)
  - L2 regularization (see tf.nn.l2_loss)
  - gradient clipping (see tf.clip_by_value or tf.clip_by_norm)
- model selection:
  - early stopping
- hyper-parameter optimization (e.g. random search or grid search (expensive!))
    - initial learning rate
    - dropout probability
    - input and output size
    - L2 regularization
    - gradient clipping value
    - batch size
    - ...
- post-processing
  - for incorporating consistency constraints

In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
#! SETUP 1 - DO NOT CHANGE, MOVE NOR COPY
import sys, os
_snlp_book_dir = "../../../../../"
sys.path.append(_snlp_book_dir)
# docker image contains tensorflow 0.10.0rc0. We will support execution of only that version!
import statnlpbook.nn as nn

import tensorflow as tf
import numpy as np

In [2]:
#! SETUP 2 - DO NOT CHANGE, MOVE NOR COPY
data_path = _snlp_book_dir + "data/nn/"
data_train = nn.load_corpus(data_path + "train.tsv")
data_dev = nn.load_corpus(data_path + "dev.tsv")
assert(len(data_train) == 45502)

In [3]:
def tokenize(input):
    return input.split(' ')#.replace("'s"," 's").replace('.',' .').replace(',',' ,').replace('?',' ?').replace('!',' !').split(' ')

def pipeline(data, vocab=None, max_sent_len_=None):
    is_ext_vocab = True
    if vocab is None:
        is_ext_vocab = False
        vocab = {'<PAD>': 0, '<OOV>': 1}

    max_sent_len = -1
    data_sentences = []
    data_orders = []
    for instance in data:
        sents = []
        for sentence in instance['story']:
            sent = []
            tokenized = tokenize(sentence)
            for token in tokenized:
                if not is_ext_vocab and token not in vocab:
                    vocab[token] = len(vocab)
                if token not in vocab:
                    token_id = vocab['<OOV>']
                else:
                    token_id = vocab[token]
                sent.append(token_id)
            if len(sent) > max_sent_len:
                max_sent_len = len(sent)
            sents.append(sent)
        data_sentences.append(sents)
        data_orders.append(instance['order'])

    if max_sent_len_ is not None:
        max_sent_len = max_sent_len_
    out_sentences = np.full([len(data_sentences), 5, max_sent_len], vocab['<PAD>'], dtype=np.int32)

    for i, elem in enumerate(data_sentences):
        for j, sent in enumerate(elem):
            out_sentences[i, j, 0:len(sent)] = sent

    out_orders = np.array(data_orders, dtype=np.int32)
    return out_sentences, out_orders, vocab


In [4]:
# convert train set to integer IDs
train_stories, train_orders, vocab = pipeline(data_train)

In [5]:
# get the length of the longest sentence
max_sent_len = train_stories.shape[2]

# convert dev set to integer IDs, based on the train vocabulary and max_sent_len
dev_stories, dev_orders, _ = pipeline(data_dev, vocab=vocab, max_sent_len_=max_sent_len)

You can take a look at the result of the `pipeline` with the `show_data_instance` function to make sure that your data loaded correctly:

In [6]:
### MODEL PARAMETERS ###
target_size = 5
vocab_size = len(vocab)
input_size = 50
# n = len(train_stories)
output_size = 5

In [7]:
import collections
import operator
import random

file= open(data_path+'glove.twitter.27B.50d.txt', 'r', encoding='utf-8')
lines = file.readlines()
word_dict = {line.split(' ')[0]: list(map(float, line.rstrip('\n').split(' ')[1:]))  
             for line in lines if line.split(' ')[0] in vocab}
    
embedding_list=[]
a = 0
for item in vocab.items():
    if item[0]== '<PAD>':
        embedding_list.append(np.array([0 for i in range(input_size)], dtype='f'))
    elif item[0] in word_dict:
        embedding_list.append(word_dict[item[0]])
    elif item[0].lower() in word_dict:
        embedding_list.append([i+1 for i in word_dict[item[0].lower()]])
    elif item[0].rstrip("'s") in word_dict:
        embedding_list.append([i+0.6 for i in word_dict[item[0].rstrip("'s")]])
    elif item[0].rstrip(".") in word_dict:
        embedding_list.append([i+0.2 for i in word_dict[item[0].rstrip(".")]])
    elif item[0].rstrip("?") in word_dict:
        embedding_list.append([i-1 for i in word_dict[item[0].rstrip("?")]])
    elif item[0].rstrip("!") in word_dict:
        embedding_list.append([i-0.2 for i in word_dict[item[0].rstrip("!")]])
    elif item[0].rstrip(",") in word_dict:
        embedding_list.append([i-0.6 for i in word_dict[item[0].rstrip(",")]])
    elif item[0].rstrip("n't") in word_dict:
        embedding_list.append([i+0.5 for i in word_dict[item[0].rstrip("n't")]])
    else:
        a += 1
        embedding_list.append([random.uniform(-0.5, 0.5) for i in range(input_size)])
            
W = np.array(embedding_list)#[:vocab_size]

FileNotFoundError: [Errno 2] No such file or directory: '../../../../../data/nn/glove.twitter.27B.50d.txt'

In [8]:
a

14509

In [12]:
len(W)

53646

In [9]:
### MODEL ###

## PLACEHOLDERS
story = tf.placeholder(tf.int64, [None, None, None], "story")        # [batch_size x 5 x max_length]
order = tf.placeholder(tf.int64, [None, None], "order")              # [batch_size x 5]

batch_size = tf.shape(story)[0]

sentences = [tf.reshape(x, [batch_size, -1]) for x in tf.split(axis=1, num_or_size_splits=5, value=story)]  # 5 times [batch_size x max_length]

# Word embeddings
#initializer = tf.random_uniform_initializer(-0.1, 0.1)
#embeddings = tf.get_variable("W", [vocab_size, input_size], initializer=initializer)
initializer = tf.constant(W) 
embeddings = tf.get_variable("a", initializer=initializer)

sentences_embedded = [tf.nn.embedding_lookup(embeddings, sentence)   # [batch_size x max_seq_length x input_size]
                      for sentence in sentences]

hs = [tf.reduce_sum(sentence, 1) for sentence in sentences_embedded] # 5 times [batch_size x input_size]

h = tf.concat(axis=1, values=hs)    # [batch_size x 5*input_size]
h = tf.reshape(h, [batch_size, 5*input_size])

logits_flat = tf.contrib.layers.linear(h, 5 * target_size)    # [batch_size x 5*target_size]
logits = tf.reshape(logits_flat, [-1, 5, target_size])        # [batch_size x 5 x target_size]

# loss 
loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=order))

# prediction function
unpacked_logits = [tensor for tensor in tf.unstack(logits, axis=1)]
softmaxes = [tf.nn.softmax(tensor) for tensor in unpacked_logits]
softmaxed_logits = tf.stack(softmaxes, axis=1)
predict = tf.arg_max(softmaxed_logits, 2)

In [ ]:
opt_op = tf.train.AdamOptimizer(0.001).minimize(loss)

In [ ]:
BATCH_SIZE = 25

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    n = train_stories.shape[0]

    for epoch in range(5):
        print('----- Epoch', epoch, '-----')
        total_loss = 0
        for i in range(n // BATCH_SIZE):
            inst_story = train_stories[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
            inst_order = train_orders[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
            feed_dict = {story: inst_story, order: inst_order}
            _, current_loss = sess.run([opt_op, loss], feed_dict=feed_dict)
            total_loss += current_loss

        print(' Train loss:', total_loss / n)

        train_feed_dict = {story: train_stories, order: train_orders}
        train_predicted = sess.run(predict, feed_dict=train_feed_dict)
        train_accuracy = nn.calculate_accuracy(train_orders, train_predicted)
        print(' Train accuracy:', train_accuracy)
        
        dev_feed_dict = {story: dev_stories, order: dev_orders}
        dev_predicted = sess.run(predict, feed_dict=dev_feed_dict)
        dev_accuracy = nn.calculate_accuracy(dev_orders, dev_predicted)
        print(' Dev accuracy:', dev_accuracy)

        
    
    nn.save_model(sess)

----- Epoch 0 -----
 Train loss: 8.40615991883


In [12]:
# LOAD THE DATA
data_test = nn.load_corpus(data_path + "dev.tsv")
# make sure you process this with the same pipeline as you processed your dev set
test_stories, test_orders, _ = nn.pipeline(data_test, vocab=vocab, max_sent_len_=max_sent_len)

# THIS VARIABLE MUST BE NAMED `test_feed_dict`
test_feed_dict = {story: test_stories, order: test_orders}

In [13]:
#! ASSESSMENT 1 - DO NOT CHANGE, MOVE NOR COPY
with tf.Session() as sess:
    # LOAD THE MODEL
    saver = tf.train.Saver()
    saver.restore(sess, './model/model.checkpoint')
    
    # RUN TEST SET EVALUATION
    dev_predicted = sess.run(predict, feed_dict=test_feed_dict)
    dev_accuracy = nn.calculate_accuracy(dev_orders, dev_predicted)

dev_accuracy

0.49909139497594868